In [9]:
import spacy 
import pyArango
import os
from os import path
import time
import glob
import pandas as pd
from arango import ArangoClient

Lecture du texte depuis le chemin

In [2]:
def get_text(path):
    with open(path, encoding='utf8') as f:
        return(f.read().replace('\n',' '))
        f.close()

In [3]:
dir_path = os.getcwd()
dir_path

'/home/paul/projects/text explorer'

Liste des fichiers présents dans le dossier choisi pour l'analyse

In [4]:
files = glob.glob('/home/paul/projects/text_for_app/*.{}'.format('txt'))
files

['/home/paul/projects/text_for_app/jean_blog.txt',
 '/home/paul/projects/text_for_app/emploi étudiant et inégalités sociales.txt']

Obtention du nom du fichier depuis le chemin

In [5]:
def get_filename_from_path(path):
    return os.path.normpath(path).split(os.sep)[-1]

In [6]:
documents = pd.DataFrame({'filepath':files,
                          'doc_name':[get_filename_from_path(filepath) for filepath in files],
                          'doc_number':list(range(0,len(files)))})
documents

,filepath,doc_name,doc_number
0,/home/paul/projects/text_for_app/jean_blog.txt,jean_blog.txt,0
1,/home/paul/projects/text_for_app/emploi étudia...,emploi étudiant et inégalités sociales.txt,1


Initialisation de la connection avec la base de données 

In [10]:
# Initialize the client for ArangoDB.
client = ArangoClient(hosts="http://localhost:8529")

In [11]:
# Connect to "test" database as root user.
db = client.db("text_graph_test", username="root", password="root")

Création du graphe de l'application

In [13]:
graph = db.create_graph('text_explorer')

Création des collections 

In [14]:
docs = graph.create_vertex_collection('docs')
sentences = graph.create_vertex_collection('sentences')
tokens = graph.create_vertex_collection('tokens')
lemmas = graph.create_vertex_collection('lemmas')

Création des liens 

In [15]:
is_from = graph.create_edge_definition(
    edge_collection='is_from',
    from_vertex_collections=['sentences'],
    to_vertex_collections=['docs']
)

In [16]:
contracts_to = graph.create_edge_definition(
    edge_collection='contracts_to',
    from_vertex_collections=['tokens'],
    to_vertex_collections=['lemmas']
)

In [17]:
syntagmatic_link = graph.create_edge_definition(
    edge_collection='syntagmatic_link',
    from_vertex_collections=['tokens'],
    to_vertex_collections=['tokens']
)

##  l'ajout de textes au fur et à mesure n'est pas encore pris en compte 

L'objectif maintenant est de remplir ces champs avec les données extraites depuis le texte

### insertion des documents

In [22]:
for doc_name, number in zip(documents['doc_name'], documents['doc_number']):
    docs.insert({'_key':f'doc{number}',
                 'doc_name':doc_name})

Chargement du modèle

In [23]:
nlp = spacy.load('fr_core_news_lg')

Traitement des fichiers par le modèle

## se renseigner sur comment faire du traitement en batch
    - mesurer l'empreinte mémoire de l'opération et ajuster combien de documents en même temps peuvent être traités

In [25]:
processed_docs = list(nlp.pipe([get_text(path) for path in documents['filepath']]))

### insertion des phrases

In [35]:
for doc in processed_docs:
    for sentence in doc.sents:
        print(sentence)

Je ne vais pas faire inutilement durer le suspense : la réponse est oui.
En tout cas sur moi : tout pantalon qui ne moule pas les fesses me fait une silhouette hideuse et me donne l’air d’un sac négligé.
 
Or le non moulage de fesses est le propre du jean mom, je ne comprends même pas comment on a pu inventer un truc pareil.
 
Et bien néanmoins, j’en possède un depuis une semaine et je ne l’ai pas quitté depuis.
Il me fait une silhouette hideuse comme prévu (en plus il est baggy, ce qui empire le truc
– oui oui, il est à ma taille ;-), mais par quelque aberration perverse, j’aime beaucoup le porter.
 
C’est peut-être pour faire un pied de nez aux diktats de la sexitude que je réprouve depuis toujours ?
 
C’est peut-être parce que de plus en plus, je porte un vêtement pour le style du vêtement lui-même et pas pour mettre mon corps en valeur au sens traditionnel du terme ?
C’est-à-dire que le sujet principal c’est le vêtement, pas moi : de plus en plus souvent, le plus important pour moi

###

Extraction d'une forme tabulaire de la structure syntagmatique.
  > Le processus est itéré sur l'ensemble des phrases du document

on obtient une liste de dataframes contenant la structure syntagmatique pour chaque phrase

In [18]:
def create_dependancy_df_list(processed_text):
    df_list = []
    for sentence in processed_text.sents:
        token_text, token_dep, token_head_text, token_head_pos = [], [], [], []
        for token in sentence:
            if not token.is_punct and not token.is_stop and not token.is_space:
                token_text.append(token.text)
                token_dep.append(token.dep_), 
                token_head_text.append(token.head.text), 
                token_head_pos.append( token.head.pos_)
        df = pd.DataFrame({'token':token_text,
                           'dep':token_dep,
                           'head_text':token_head_text,
                           'head_pos':token_head_pos})    
        if not df.empty:
            df_list.append(df)
        else:
            pass
    return df_list

In [19]:
file_1_processed = nlp(get_text(documents['filepath'][0]))
file_2_processed = nlp(get_text(documents['filepath'][1]))

In [20]:
create_dependancy_df_list(file_1_processed)[0]

,token,dep,head_text,head_pos
0,faire,xcomp,vais,VERB
1,inutilement,advmod,faire,VERB
2,durer,xcomp,faire,VERB
3,suspense,obj,durer,VERB
4,réponse,nsubj,oui,ADV
5,oui,parataxis,vais,VERB
